In [ ]:
import os
import requests
import pandas as pd
import numpy as np

from pystac_client import Client
from shapely.geometry import shape, Point

import geopandas as gpd
import matplotlib.pyplot as plt


from datetime import timedelta

from planetary_computer import sign
from stackstac import stack
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import datetime

In [ ]:
ais = pd.read_parquet('data/gulf_jan/intersections.parquet').drop(columns='geometry')

In [ ]:
from pystac_client import Client
api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


In [ ]:
ais.sample()

In [ ]:
# Choose a row index from your AIS dataframe
ais_idx = 78787  # <- change this to whatever row you're investigating
row = ais.iloc[ais_idx]
print(f"Selected MMSI: {row['MMSI']} @ {row['BaseDateTime']}")

# Define point and time buffer
ais_point = Point(row["LON"], row["LAT"])
time_buffer = timedelta(minutes=30)

# Load image with a slightly bigger bbox (0.2° instead of 0.1°)
search = api.search(
    collections=["sentinel-2-l2a"],
    bbox=[row["LON"] - 0.1, row["LAT"] - 0.1, row["LON"] + 0.1, row["LAT"] + 0.1],
    datetime=f"{(row['BaseDateTime'] - time_buffer).isoformat()}/{(row['BaseDateTime'] + time_buffer).isoformat()}",
    query={"eo:cloud_cover": {"lt": 80}},
    limit=5
)

items = list(search.get_items())


tile_geom = shape(items[0].geometry)
ais_point = Point(row["LON"], row["LAT"])

print("Tile bounds:", tile_geom.bounds)
print("AIS point inside tile?", tile_geom.contains(ais_point))


if not items:
    print("No Sentinel-2 tile found.")
else:
    item = items[0]
    signed_item = sign(item)

    rgb_bands = ["B04", "B03", "B02"]
    arr = stack([signed_item], assets=rgb_bands, epsg=4326, resolution=10)

    # Crop around AIS point
    buffer_deg = 0.01  # about 1 km at equator
    cropped = arr.sel(
        x=slice(row["LON"] - buffer_deg, row["LON"] + buffer_deg),
        y=slice(row["LAT"] + buffer_deg, row["LAT"] - buffer_deg)  # lat reversed (north to south)
    )

    # Reshape and display
    rgb = cropped.sel(band=rgb_bands).isel(time=0).transpose("band", "y", "x")
    img = rgb.transpose("y", "x", "band").values

    if img.size == 0:
        print("Image crop returned no data (outside valid bounds).")
    else:
        vmin, vmax = np.percentile(img[~np.isnan(img)], (2, 98))
        img_scaled = np.clip((img - vmin) / (vmax - vmin), 0, 1)

        plt.figure(figsize=(8, 8))
        plt.imshow(img_scaled)
        plt.title(f"Ship @ {row['LAT']:.2f}, {row['LON']:.2f}")
        plt.axis("off")
        plt.show()

In [ ]:

# Normalize and display with percentile stretch
img_data = rgb.transpose("y", "x", "band").values

# Compute robust percentiles
vmin = np.percentile(img_data, 2)
vmax = np.percentile(img_data, 98)

# Rescale
img_scaled = np.clip((img_data - vmin) / (vmax - vmin), 0, 1)

plt.figure(figsize=(10, 10))
plt.imshow(img_scaled)
plt.title(f"MMSI {row['MMSI']} @ {row['BaseDateTime']}")
plt.axis("off")
plt.show()

In [ ]:
print(rgb.shape, rgb.dtype, np.min(img_data), np.max(img_data))
